### Supertonic-2

In [23]:
from collections import deque

total_messages = deque(maxlen=10)
messages = [{"role": "user", "content": "안녕?"}]
messages1 = [{"role": "user", "content": "안녕?"}]
total_messages.append(messages)
total_messages.append(messages1)
total_messages

deque([[{'role': 'user', 'content': '안녕?'}],
       [{'role': 'user', 'content': '안녕?'}]],
      maxlen=10)

In [24]:
list(total_messages)

[[{'role': 'user', 'content': '안녕?'}], [{'role': 'user', 'content': '안녕?'}]]

In [24]:
import sys
sys.path.append("../supertonic/py")

from helper import load_text_to_speech, timer, sanitize_filename, load_voice_style
import soundfile as sf
from IPython.display import Audio

In [43]:
# 모델 로드
onnx_dir = "../supertonic/py/assets/supertonic-2/onnx"
text_to_speech = load_text_to_speech(onnx_dir, use_gpu=False)

# 화자 스타일 로드
style_dir = "../supertonic/assets/voice_styles/F1.json"
style = load_voice_style([style_dir], verbose=True)

# 더미 파일 생성
_ = text_to_speech("테스트", "ko", style, total_step=5, speed=1.3)

Using CPU for inference
Loaded 1 voice styles


In [45]:
text = "대전광역시청의 주소는 대전광역시 서구 내방로 111입니다."
lang = "ko"

wav, duration = text_to_speech(text, lang, style, total_step=5, speed=1.2)

output_path = './audio/supertonic_tts.wav'
sf.write(output_path, wav[0, :int(text_to_speech.sample_rate * duration[0].item())], text_to_speech.sample_rate)

Audio(output_path)

### MeloTTS

In [1]:
from memory_monitor import MemoryMonitor
import torch, psutil, os, gc, re, json
import sys
sys.path.append("/home/user/work/MeloTTS")

from melo.api import TTS
import torch
from IPython.display import Audio

/home/user/miniconda3/envs/tts_hori/lib/python3.10/site-packages/librosa/util/files.py:10: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_filename
/home/user/miniconda3/envs/tts_hori/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/user/miniconda3/envs/tts_hori/lib/python3.10/site-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/user/miniconda3/envs/tts_hori/lib/pyt

In [2]:
monitor = MemoryMonitor(device_index=0)
torch.cuda.empty_cache()
gc.collect()

monitor.print_usage(label="Before Loading")

speed = 1.2
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
output_path = './audio/melo_tts.wav'

model = TTS(language='KR', device=device)

speaker_ids = model.hps.data.spk2id

# 더미 파일 생성
model.tts_to_file("테스트", speaker_ids['KR'], output_path, speed=speed)

monitor.print_usage(label="After Loading")

[Before Loading] RAM: 1143.46 MB | VRAM: 0.00 MB


/home/user/miniconda3/envs/tts_hori/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


 > Text split to sentences.
테스트
 > ===========================


  0%|          | 0/1 [00:00<?, ?it/s]/home/user/miniconda3/envs/tts_hori/lib/python3.10/site-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at kykim/bert-kor-base were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/home/user/miniconda

[After Loading] RAM: 2272.81 MB | VRAM: 0.00 MB


In [49]:
text = "대전광역시청의 주소는 대전광역시 서구 내방로 백십일입니다."
model.tts_to_file(text, speaker_ids['KR'], output_path, speed=speed)

Audio(output_path)

monitor.print_usage(label="After Loading")

 > Text split to sentences.
대전광역시청의 주소는 대전광역시 서구 내방로 백십일입니다.
 > ===========================


100%|██████████| 1/1 [00:00<00:00,  1.57it/s]

[After Loading] RAM: 2515.43 MB | VRAM: 0.00 MB


In [46]:
import re

full_response = """<function=delete_alarm>{"hour": 15, "message": "당뇨약"}</function><function=set_alarms>{"hour": 16, "message": "당뇨약", "relative_day": 0}</function>"""

pattern = r'<function=(\w+)>(.+?)</function>'
matches = re.finditer(pattern, full_response)
has_called_tool = False
tool_calls_found = list(matches)

In [47]:
for match in tool_calls_found:
    print(f"함수명: {match.group(1)}")
    print(f"인자값: {match.group(2)}")  # 여기서 확인하면 "당뇨약"이 다 나옵니다!

함수명: delete_alarm
인자값: {"hour": 15, "message": "당뇨약"}
함수명: set_alarms
인자값: {"hour": 16, "message": "당뇨약", "relative_day": 0}
